In [801]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier


from sklearn.metrics import mean_squared_error,f1_score


from sklearn.svm import SVR,LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [802]:
df = pd.read_excel('Clean_data.xlsx')

pd.set_option('display.max_columns',None)
pd.set_option("display.max_rows", None)

In [803]:

df.head()

,Survey S/N,Gender,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Co -Morbidities / Diseases,Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp,MFI_1,MFI_2,MFI_3,MFI_4,MFI_5,MFI_6,MFI_7,MFI_8,MFI_9,MFI_10,MFI_11,MFI_12,MFI_13,MFI_14,MFI_15,MFI_16,MFI_17,MFI_18,MFI_19,MFI_20,General_fat,Physical_fat,Mental_fat,Reduced_fat,Reduced_activity
0,1,1,0,48.0,13.8,31,1.69,63,No,No,60,45,0,3,9.0,No,No,No,DK,DK,No,No,No,No,No,No,No,No,2,1,1,1,4,1,2,1,1,1,2,3,3,1,1,3,1,1,3,2,3,1,1,1,2
1,2,1,0,161.0,14.2,49,1.60,52,No,No,60,0,0,3,0.0,No,Yes,No,No,No,No,No,No,No,No,No,No,No,2,2,2,2,2,2,2,3,1,1,2,3,4,1,2,1,1,1,4,2,2,2,1,1,3
2,3,1,0,12.0,14.9,27,1.59,51,No,Yes,60,0,0,2,12.0,No,DK,DK,Yes,DK,Yes,DK,No,No,No,No,No,No,1,1,2,1,5,1,1,1,1,1,2,3,1,1,1,3,1,1,3,2,3,1,1,1,1
3,4,1,0,3.0,7.9,46,1.49,54,Hypothyroid,Yes,60,0,0,3,12.0,Yes,Yes,DK,DK,No,Yes,No,Yes,No,No,Yes,Yes,Yes,3,1,3,2,1,3,3,3,1,1,3,2,1,1,2,2,1,1,1,3,2,2,1,2,2
4,5,1,0,140.0,15.4,57,1.58,55,No,No,160,0,0,3,0.0,No,No,No,No,No,Yes,No,No,No,No,No,No,No,2,2,1,1,4,1,1,2,4,2,1,2,1,1,1,2,1,1,1,2,2,1,1,1,1


In [804]:
'''def f(row):
    if row['Ferritin (ug/L)'] > 15 or row['Ferritin (ug/L)'] < 30:
        val = 1
    elif row['Ferritin (ug/L)'] < 15:
        val = 0
    else:
        val = -1
    return val

df['IS'] = df['Ferritin (ug/L)'].apply(f,axis=1)'''

# create a list of our conditions
conditions = [
    (df['Ferritin (ug/L)'] < 16),
    (df['Ferritin (ug/L)'] > 15) & (df['Ferritin (ug/L)'] < 30),
    (df['Ferritin (ug/L)'] > 30),
    ]

values = ['High Risk', 'Medium Risk','Low Risk']

df['Target'] = np.select(conditions, values)

'''
df['Target'] = np.where(
    df['Ferritin (ug/L)'] < 15, 0, np.where(
    df['Ferritin (ug/L)'] >  15 , 1, -1)) '''




#df['race_label'] = df.apply (lambda row: label_race(row), axis=1)

"\ndf['Target'] = np.where(\n    df['Ferritin (ug/L)'] < 15, 0, np.where(\n    df['Ferritin (ug/L)'] >  15 , 1, -1)) "

In [805]:
df.head()

,Survey S/N,Gender,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Co -Morbidities / Diseases,Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp,MFI_1,MFI_2,MFI_3,MFI_4,MFI_5,MFI_6,MFI_7,MFI_8,MFI_9,MFI_10,MFI_11,MFI_12,MFI_13,MFI_14,MFI_15,MFI_16,MFI_17,MFI_18,MFI_19,MFI_20,General_fat,Physical_fat,Mental_fat,Reduced_fat,Reduced_activity,Target
0,1,1,0,48.0,13.8,31,1.69,63,No,No,60,45,0,3,9.0,No,No,No,DK,DK,No,No,No,No,No,No,No,No,2,1,1,1,4,1,2,1,1,1,2,3,3,1,1,3,1,1,3,2,3,1,1,1,2,Low Risk
1,2,1,0,161.0,14.2,49,1.60,52,No,No,60,0,0,3,0.0,No,Yes,No,No,No,No,No,No,No,No,No,No,No,2,2,2,2,2,2,2,3,1,1,2,3,4,1,2,1,1,1,4,2,2,2,1,1,3,Low Risk
2,3,1,0,12.0,14.9,27,1.59,51,No,Yes,60,0,0,2,12.0,No,DK,DK,Yes,DK,Yes,DK,No,No,No,No,No,No,1,1,2,1,5,1,1,1,1,1,2,3,1,1,1,3,1,1,3,2,3,1,1,1,1,High Risk
3,4,1,0,3.0,7.9,46,1.49,54,Hypothyroid,Yes,60,0,0,3,12.0,Yes,Yes,DK,DK,No,Yes,No,Yes,No,No,Yes,Yes,Yes,3,1,3,2,1,3,3,3,1,1,3,2,1,1,2,2,1,1,1,3,2,2,1,2,2,High Risk
4,5,1,0,140.0,15.4,57,1.58,55,No,No,160,0,0,3,0.0,No,No,No,No,No,Yes,No,No,No,No,No,No,No,2,2,1,1,4,1,1,2,4,2,1,2,1,1,1,2,1,1,1,2,2,1,1,1,1,Low Risk


In [806]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0    Survey S/N                    777 non-null    int64  
 1   Gender                         777 non-null    int64  
 2   Iron Saturation (%)            777 non-null    int64  
 3   Ferritin (ug/L)                777 non-null    float64
 4   HB (g/dL)                      777 non-null    float64
 5   Age                            777 non-null    int64  
 6   Height
(metres)                777 non-null    float64
 7   Weight
(kg)                    777 non-null    int64  
 8   Co -Morbidities / Diseases     777 non-null    object 
 9   Premenopausal                  777 non-null    object 
 10  Running
Time (minutes)         777 non-null    int64  
 11  Cycling
Time (minutes)         777 non-null    int64  
 12  Swimming
Time (minutes)        777 non-null    int

## preprocessing

In [807]:
{column : len(df[column].unique()) for column in df.columns }

{' Survey S/N': 772,
 'Gender': 1,
 'Iron Saturation (%)': 53,
 'Ferritin (ug/L)': 200,
 'HB (g/dL)': 80,
 'Age': 54,
 'Height\n(metres)': 46,
 'Weight\n(kg)': 66,
 'Co -Morbidities / Diseases': 32,
 'Premenopausal': 2,
 'Running\nTime (minutes)': 35,
 'Cycling\nTime (minutes)': 21,
 'Swimming\nTime (minutes)': 15,
 'Competition Level (1-6)': 7,
 'Periods (female only)': 20,
 'Period Advice\n(female only)': 3,
 'Anaemia': 3,
 'Anaemia last 3 months': 3,
 'Iron Deficient': 3,
 'Iron Deficient last 3 months': 3,
 'Iron Supplement': 3,
 'Iron Supplement last 3 months': 3,
 'Heavy Menstrual Bleeding \n1': 2,
 'Heavy Menstrual Bleeding \n2': 2,
 'Heavy Menstrual Bleeding \n3': 2,
 'Heavy Menstrual Bleeding \n4': 2,
 'HMB? (Yes/No)': 2,
 'Heavy Menstrual Bleeding_Imp': 2,
 'MFI_1': 5,
 'MFI_2': 6,
 'MFI_3': 5,
 'MFI_4': 5,
 'MFI_5': 5,
 'MFI_6': 5,
 'MFI_7': 5,
 'MFI_8': 5,
 'MFI_9': 5,
 'MFI_10': 5,
 'MFI_11': 6,
 'MFI_12': 5,
 'MFI_13': 5,
 'MFI_14': 5,
 'MFI_15': 5,
 'MFI_16': 5,
 'MFI_17

In [808]:
categorical_cols = df.select_dtypes('object').columns
cat = df[categorical_cols]
print(categorical_cols)
{column : len(cat[column].unique()) for column in cat.columns }

Index(['Co -Morbidities / Diseases', 'Premenopausal',
       'Period Advice\n(female only)', 'Anaemia', 'Anaemia last 3 months',
       'Iron Deficient', 'Iron Deficient last 3 months', 'Iron Supplement',
       'Iron Supplement last 3 months', 'Heavy Menstrual Bleeding \n1',
       'Heavy Menstrual Bleeding \n2', 'Heavy Menstrual Bleeding \n3',
       'Heavy Menstrual Bleeding \n4', 'HMB? (Yes/No)',
       'Heavy Menstrual Bleeding_Imp', 'Target'],
      dtype='object')


{'Co -Morbidities / Diseases': 32,
 'Premenopausal': 2,
 'Period Advice\n(female only)': 3,
 'Anaemia': 3,
 'Anaemia last 3 months': 3,
 'Iron Deficient': 3,
 'Iron Deficient last 3 months': 3,
 'Iron Supplement': 3,
 'Iron Supplement last 3 months': 3,
 'Heavy Menstrual Bleeding \n1': 2,
 'Heavy Menstrual Bleeding \n2': 2,
 'Heavy Menstrual Bleeding \n3': 2,
 'Heavy Menstrual Bleeding \n4': 2,
 'HMB? (Yes/No)': 2,
 'Heavy Menstrual Bleeding_Imp': 2,
 'Target': 4}

In [809]:
num_cols = df.select_dtypes(['int64','float64']).columns
num = df[num_cols]

In [810]:
df['Period Advice\n(female only)'].value_counts()

No     697
Yes     76
DK       4
Name: Period Advice\n(female only), dtype: int64

In [811]:
#for cols in df.columns.drop(['Co -Morbidities / Diseases',' Survey S/N','Target']):
#    df[cols] = df[cols].replace({"Yes":1,"No":0,"DK":2})
    
    
#for cols in cat:
#    cat[cols] = cat[cols].replace({"Yes":1,"No":0,"DK":2})
#df = df.columns.drop('Co -Morbidities / Diseases')
df = df.drop('Co -Morbidities / Diseases', 1)
df.head()

,Survey S/N,Gender,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp,MFI_1,MFI_2,MFI_3,MFI_4,MFI_5,MFI_6,MFI_7,MFI_8,MFI_9,MFI_10,MFI_11,MFI_12,MFI_13,MFI_14,MFI_15,MFI_16,MFI_17,MFI_18,MFI_19,MFI_20,General_fat,Physical_fat,Mental_fat,Reduced_fat,Reduced_activity,Target
0,1,1,0,48.0,13.8,31,1.69,63,No,60,45,0,3,9.0,No,No,No,DK,DK,No,No,No,No,No,No,No,No,2,1,1,1,4,1,2,1,1,1,2,3,3,1,1,3,1,1,3,2,3,1,1,1,2,Low Risk
1,2,1,0,161.0,14.2,49,1.60,52,No,60,0,0,3,0.0,No,Yes,No,No,No,No,No,No,No,No,No,No,No,2,2,2,2,2,2,2,3,1,1,2,3,4,1,2,1,1,1,4,2,2,2,1,1,3,Low Risk
2,3,1,0,12.0,14.9,27,1.59,51,Yes,60,0,0,2,12.0,No,DK,DK,Yes,DK,Yes,DK,No,No,No,No,No,No,1,1,2,1,5,1,1,1,1,1,2,3,1,1,1,3,1,1,3,2,3,1,1,1,1,High Risk
3,4,1,0,3.0,7.9,46,1.49,54,Yes,60,0,0,3,12.0,Yes,Yes,DK,DK,No,Yes,No,Yes,No,No,Yes,Yes,Yes,3,1,3,2,1,3,3,3,1,1,3,2,1,1,2,2,1,1,1,3,2,2,1,2,2,High Risk
4,5,1,0,140.0,15.4,57,1.58,55,No,160,0,0,3,0.0,No,No,No,No,No,Yes,No,No,No,No,No,No,No,2,2,1,1,4,1,1,2,4,2,1,2,1,1,1,2,1,1,1,2,2,1,1,1,1,Low Risk


In [812]:
'''def preprocess_inputs(df,num_cols):
    z = df.copy()
    z['Gender'] = z['Gender'].replace({"F":1,"M":0})
    for cols in z.columns.drop(num_cols):
        z[cols] = z[cols].replace({"Yes":1,"No":0,"DK":2})
        
    y = z['Target']
    X = z.drop('Target',axis=1)
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,schuffle=True,random_state = 2)
    
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train),index = X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test),index = X_train.index, columns = X_test.columns)
    
    
    return X_train,X_test,y_train,y_test'''

'def preprocess_inputs(df,num_cols):\n    z = df.copy()\n    z[\'Gender\'] = z[\'Gender\'].replace({"F":1,"M":0})\n    for cols in z.columns.drop(num_cols):\n        z[cols] = z[cols].replace({"Yes":1,"No":0,"DK":2})\n        \n    y = z[\'Target\']\n    X = z.drop(\'Target\',axis=1)\n    \n    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,schuffle=True,random_state = 2)\n    \n    \n    scaler = StandardScaler()\n    scaler.fit(X_train)\n    X_train = pd.DataFrame(scaler.transform(X_train),index = X_train.index, columns = X_train.columns)\n    X_test = pd.DataFrame(scaler.transform(X_test),index = X_train.index, columns = X_test.columns)\n    \n    \n    return X_train,X_test,y_train,y_test'

In [813]:
#df['Gender'] = df['Gender'].replace({"F":1,"M":0})
for cols in df.columns.drop(num_cols):
    df[cols] = df[cols].replace({"Yes":1,"No":0,"DK":2})
df.head()

,Survey S/N,Gender,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp,MFI_1,MFI_2,MFI_3,MFI_4,MFI_5,MFI_6,MFI_7,MFI_8,MFI_9,MFI_10,MFI_11,MFI_12,MFI_13,MFI_14,MFI_15,MFI_16,MFI_17,MFI_18,MFI_19,MFI_20,General_fat,Physical_fat,Mental_fat,Reduced_fat,Reduced_activity,Target
0,1,1,0,48.0,13.8,31,1.69,63,0,60,45,0,3,9.0,0,0,0,2,2,0,0,0,0,0,0,0,0,2,1,1,1,4,1,2,1,1,1,2,3,3,1,1,3,1,1,3,2,3,1,1,1,2,Low Risk
1,2,1,0,161.0,14.2,49,1.60,52,0,60,0,0,3,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,1,1,2,3,4,1,2,1,1,1,4,2,2,2,1,1,3,Low Risk
2,3,1,0,12.0,14.9,27,1.59,51,1,60,0,0,2,12.0,0,2,2,1,2,1,2,0,0,0,0,0,0,1,1,2,1,5,1,1,1,1,1,2,3,1,1,1,3,1,1,3,2,3,1,1,1,1,High Risk
3,4,1,0,3.0,7.9,46,1.49,54,1,60,0,0,3,12.0,1,1,2,2,0,1,0,1,0,0,1,1,1,3,1,3,2,1,3,3,3,1,1,3,2,1,1,2,2,1,1,1,3,2,2,1,2,2,High Risk
4,5,1,0,140.0,15.4,57,1.58,55,0,160,0,0,3,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,1,4,1,1,2,4,2,1,2,1,1,1,2,1,1,1,2,2,1,1,1,1,Low Risk


In [814]:
y = df['Target']
X = df.drop('Target',axis=1)
    
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,random_state = 2)
    
    
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [815]:
#X_train,X_test,y_train,y_test = preprocess_inputs(df,num_cols)

In [816]:
#X_train.describe()

## Training

In [817]:
models={
    "LogisticRegression" : LogisticRegression(),
    "KNeighborsClassifier" : KNeighborsClassifier(),
    "DecisionTreeClassifier" : DecisionTreeClassifier(),
    "LinearSVC" : LinearSVC(),
    "SVC" : SVC(),
    "MLPClassifier" : MLPClassifier(),
    "RandomForestClassifier" : RandomForestClassifier(),
    "GradientBoostingClassifier" : GradientBoostingClassifier(),
}

In [818]:
for name,model in models.items():
    model.fit(X_train,y_train)
    print(name + "Model Trained.")
    

LogisticRegressionModel Trained.
KNeighborsClassifierModel Trained.
DecisionTreeClassifierModel Trained.


/opt/homebrew/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVCModel Trained.
SVCModel Trained.
MLPClassifierModel Trained.
RandomForestClassifierModel Trained.
GradientBoostingClassifierModel Trained.


## Results

In [819]:
for name,model in models.items():
    print(name + ': {:.2f}%'.format(model.score(X_test,y_test) * 100))

LogisticRegression: 90.38%
KNeighborsClassifier: 83.33%
DecisionTreeClassifier: 100.00%
LinearSVC: 89.10%
SVC: 77.56%
MLPClassifier: 91.03%
RandomForestClassifier: 98.08%
GradientBoostingClassifier: 100.00%


In [820]:
for name,model in models.items():
    print(name + ': {:.2f}%'.format(model.score(X_train,y_train) * 100))

LogisticRegression: 95.33%
KNeighborsClassifier: 85.67%
DecisionTreeClassifier: 100.00%
LinearSVC: 88.73%
SVC: 72.14%
MLPClassifier: 96.14%
RandomForestClassifier: 100.00%
GradientBoostingClassifier: 100.00%


## Regression

In [821]:
df.head()

,Survey S/N,Gender,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp,MFI_1,MFI_2,MFI_3,MFI_4,MFI_5,MFI_6,MFI_7,MFI_8,MFI_9,MFI_10,MFI_11,MFI_12,MFI_13,MFI_14,MFI_15,MFI_16,MFI_17,MFI_18,MFI_19,MFI_20,General_fat,Physical_fat,Mental_fat,Reduced_fat,Reduced_activity,Target
0,1,1,0,48.0,13.8,31,1.69,63,0,60,45,0,3,9.0,0,0,0,2,2,0,0,0,0,0,0,0,0,2,1,1,1,4,1,2,1,1,1,2,3,3,1,1,3,1,1,3,2,3,1,1,1,2,Low Risk
1,2,1,0,161.0,14.2,49,1.60,52,0,60,0,0,3,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,1,1,2,3,4,1,2,1,1,1,4,2,2,2,1,1,3,Low Risk
2,3,1,0,12.0,14.9,27,1.59,51,1,60,0,0,2,12.0,0,2,2,1,2,1,2,0,0,0,0,0,0,1,1,2,1,5,1,1,1,1,1,2,3,1,1,1,3,1,1,3,2,3,1,1,1,1,High Risk
3,4,1,0,3.0,7.9,46,1.49,54,1,60,0,0,3,12.0,1,1,2,2,0,1,0,1,0,0,1,1,1,3,1,3,2,1,3,3,3,1,1,3,2,1,1,2,2,1,1,1,3,2,2,1,2,2,High Risk
4,5,1,0,140.0,15.4,57,1.58,55,0,160,0,0,3,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,1,4,1,1,2,4,2,1,2,1,1,1,2,1,1,1,2,2,1,1,1,1,Low Risk


In [822]:
y = df['Ferritin (ug/L)']
X = df.drop(['Target','Ferritin (ug/L)'],axis=1)

In [823]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,random_state = 2)
    
    
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [824]:
reg_models = {
    "Linear Regression " : LinearRegression(),
    "DecisionTreeRegression" : DecisionTreeRegressor(),
    "LinearSVR" : LinearSVR(),
    "SVR" : SVR(),
    "MLPRegressor" : MLPRegressor(),
    "RandomForestRegressor" : RandomForestRegressor(),
    "GradientBoostingRegressor" : GradientBoostingRegressor(),
}

In [825]:
for name,model in reg_models.items():
    model.fit(X_train,y_train)
    print(name + "Model Trained.")

Linear Regression Model Trained.
DecisionTreeRegressionModel Trained.
LinearSVRModel Trained.
SVRModel Trained.


/opt/homebrew/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/homebrew/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressorModel Trained.
RandomForestRegressorModel Trained.
GradientBoostingRegressorModel Trained.


In [826]:

for name,model in reg_models.items():
    y_pred = model.predict(X_train)
    #((model.predict(X_test)))
    pred = mean_squared_error(y_train, y_pred)
    print(name + 'MSE Score : ', pred)

Linear Regression MSE Score :  3357.092655001254
DecisionTreeRegressionMSE Score :  0.0
LinearSVRMSE Score :  3861.954743640325
SVRMSE Score :  5729.576675495526
MLPRegressorMSE Score :  3332.3510977860205
RandomForestRegressorMSE Score :  511.771428220612
GradientBoostingRegressorMSE Score :  982.6431519052986


In [827]:
len(df.columns)

53

## Reduced Data set

In [828]:
df.drop(df.iloc[:, -26:], inplace = True, axis = 1)

In [829]:
df.head()

,Survey S/N,Gender,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp
0,1,1,0,48.0,13.8,31,1.69,63,0,60,45,0,3,9.0,0,0,0,2,2,0,0,0,0,0,0,0,0
1,2,1,0,161.0,14.2,49,1.60,52,0,60,0,0,3,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,3,1,0,12.0,14.9,27,1.59,51,1,60,0,0,2,12.0,0,2,2,1,2,1,2,0,0,0,0,0,0
3,4,1,0,3.0,7.9,46,1.49,54,1,60,0,0,3,12.0,1,1,2,2,0,1,0,1,0,0,1,1,1
4,5,1,0,140.0,15.4,57,1.58,55,0,160,0,0,3,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [830]:
df.drop(df.iloc[:,:2],inplace = True,axis = 1)
df.head()

,Iron Saturation (%),Ferritin (ug/L),HB (g/dL),Age,Height\n(metres),Weight\n(kg),Premenopausal,Running\nTime (minutes),Cycling\nTime (minutes),Swimming\nTime (minutes),Competition Level (1-6),Periods (female only),Period Advice\n(female only),Anaemia,Anaemia last 3 months,Iron Deficient,Iron Deficient last 3 months,Iron Supplement,Iron Supplement last 3 months,Heavy Menstrual Bleeding \n1,Heavy Menstrual Bleeding \n2,Heavy Menstrual Bleeding \n3,Heavy Menstrual Bleeding \n4,HMB? (Yes/No),Heavy Menstrual Bleeding_Imp
0,0,48.0,13.8,31,1.69,63,0,60,45,0,3,9.0,0,0,0,2,2,0,0,0,0,0,0,0,0
1,0,161.0,14.2,49,1.60,52,0,60,0,0,3,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,12.0,14.9,27,1.59,51,1,60,0,0,2,12.0,0,2,2,1,2,1,2,0,0,0,0,0,0
3,0,3.0,7.9,46,1.49,54,1,60,0,0,3,12.0,1,1,2,2,0,1,0,1,0,0,1,1,1
4,0,140.0,15.4,57,1.58,55,0,160,0,0,3,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [831]:
y = df['Ferritin (ug/L)']
#df['Height\n(metres)']
#df['Weight\n(kg)']
df['HB (g/dL)']
X = df.drop(['Weight\n(kg)','Height\n(metres)','Swimming\nTime (minutes)','Ferritin (ug/L)','Competition Level (1-6)','Running\nTime (minutes)','Cycling\nTime (minutes)'],axis=1)

In [832]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,random_state = 2)
    
    
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [833]:
reg_models = {
    "Linear Regression " : LinearRegression(),
    "DecisionTreeRegression" : DecisionTreeRegressor(),
    "LinearSVR" : LinearSVR(),
    "SVR" : SVR(),
    "MLPRegressor" : MLPRegressor(),
    "RandomForestRegressor" : RandomForestRegressor(),
    "GradientBoostingRegressor" : GradientBoostingRegressor(),
}

In [834]:
for name,model in reg_models.items():
    model.fit(X_train,y_train)
    print(name + "Model Trained.")

Linear Regression Model Trained.
DecisionTreeRegressionModel Trained.
LinearSVRModel Trained.
SVRModel Trained.


/opt/homebrew/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/homebrew/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressorModel Trained.
RandomForestRegressorModel Trained.
GradientBoostingRegressorModel Trained.


In [835]:

for name,model in reg_models.items():
    y_pred = model.predict(X_train)
    #((model.predict(X_test)))
    pred = mean_squared_error(y_train, y_pred)
    print(name + 'MSE Score : ', pred)

Linear Regression MSE Score :  3731.2804922544115
DecisionTreeRegressionMSE Score :  244.95120772946862
LinearSVRMSE Score :  3986.6170292800844
SVRMSE Score :  4654.658749296865
MLPRegressorMSE Score :  3574.2485137084477
RandomForestRegressorMSE Score :  771.4067198127264
GradientBoostingRegressorMSE Score :  1773.117511324192
